In [23]:
pip install -q pandas numpy pyarrow vaderSentiment


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# SECTION 1 — Imports & Globals

import os, gzip, json, math, re
from typing import Iterator, Dict, Any, List, Tuple, Optional

import numpy as np
import pandas as pd

# Sentiment (VADER)
try:
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    _VADER = SentimentIntensityAnalyzer()
except Exception:
    _VADER = None

import matplotlib.pyplot as plt

_WORD = re.compile(r"\b\w+\b")
_FIRST_PERSON = re.compile(r"\b(i|i'm|i’ve|i've|my|me)\b", re.IGNORECASE)


In [ ]:
# SECTION 2 — Robust Reader for Amazon/Yelp formats

def _open_text(path: str):
    if path.endswith(".gz"):
        return gzip.open(path, "rt", encoding="utf-8", errors="ignore")
    return open(path, "rt", encoding="utf-8", errors="ignore")

def parse_amz_reviews(path: str, n_limit: Optional[int]=None) -> Iterator[Dict[str, Any]]:
    """
    Yields dict reviews from:
      - JSON Lines (usual Amazon format): one JSON object per line
      - OR a single large JSON array of objects
    """
    with _open_text(path) as f:
        head = f.read(2048)
        if not head:
            return
        is_array = head.lstrip().startswith("[")

    if is_array:
        with _open_text(path) as f:
            data = json.load(f)
            for i, obj in enumerate(data):
                yield obj
                if n_limit is not None and i+1 >= n_limit:
                    break
    else:
        with _open_text(path) as f:
            for i, line in enumerate(f):
                line = line.strip()
                if not line:
                    continue
                try:
                    obj = json.loads(line)
                except Exception:
                    continue
                yield obj
                if n_limit is not None and i+1 >= n_limit:
                    break


In [ ]:
# SECTION 3 — Feature Helpers (VADER, mapping, style, utils)

def _safe_vader_compound(text: str) -> float:
    """
    VADER compound in [-1,1]. Uses a global analyzer if available, else a simple fallback.
    """
    t = text or ""
    if _VADER is not None:
        try:
            return float(_VADER.polarity_scores(t)["compound"])
        except Exception:
            pass

    tl = t.lower()
    pos = sum(tl.count(w) for w in [" good", " great", " excellent", " amazing", " love", " perfect", " happy"])
    neg = sum(tl.count(w) for w in [" bad", " terrible", " awful", " hate", " poor", " broken", " angry"])
    return 0.0 if (pos + neg) == 0 else (pos - neg) / (pos + neg)

def _stars_to_unit(stars: float) -> float:
    """Map 1..5 -> [-1, 1]"""
    try:
        s = float(stars)
        return (s - 3.0) / 2.0
    except Exception:
        return np.nan

def _month_floor(ts: pd.Timestamp) -> pd.Timestamp:
    return pd.Timestamp(year=ts.year, month=ts.month, day=1, tz="UTC")

def _is_ascii_major(text) -> bool:
    if not isinstance(text, str):
        return True
    if not text:
        return True
    ascii_chars = sum(1 for ch in text if ord(ch) < 128)
    return ascii_chars >= 0.9 * len(text)

def _to_int_or_nan(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return np.nan
    if isinstance(x, str):
        x = x.replace(",", "").strip()
    try:
        return int(x)
    except Exception:
        return np.nan

def _style_feats(txt: str) -> Tuple[float, float, float]:
    """(exclam_per_char, first_person_per_token, caps_per_char)"""
    if not isinstance(txt, str) or not txt:
        return (0.0, 0.0, 0.0)
    n_chars = max(1, len(txt))
    tokens = _WORD.findall(txt)
    n_tok = max(1, len(tokens))
    exclam = txt.count("!") / n_chars
    first_person = len(_FIRST_PERSON.findall(txt)) / n_tok
    caps = sum(1 for ch in txt if ch.isupper()) / n_chars
    return exclam, first_person, caps


In [ ]:
# SECTION 4 — Build Row-Level DataFrame

def build_reviews_df(path: str, n_limit: Optional[int]=None) -> pd.DataFrame:
    rows = []
    for rec in parse_amz_reviews(path, n_limit=n_limit):
        rows.append({
            "user_id": rec.get("reviewerID") or rec.get("user_id"),
            "item_id": rec.get("asin") or rec.get("business_id"),
            "text": rec.get("reviewText") or rec.get("text"),
            "stars": rec.get("overall") or rec.get("stars"),
            "ts": pd.to_datetime(
                rec.get("unixReviewTime"), unit="s", utc=True
            ) if rec.get("unixReviewTime") is not None else (
                pd.to_datetime(rec.get("reviewTime"), utc=True, errors="coerce")
                if rec.get("reviewTime") is not None else
                pd.to_datetime(rec.get("date"), utc=True, errors="coerce")
            ),
            "summary": rec.get("summary"),
            "helpful_votes": _to_int_or_nan(rec.get("vote") or rec.get("useful")),
        })
    df = pd.DataFrame(rows)
    df = df.dropna(subset=["user_id", "item_id", "ts"]).reset_index(drop=True)
    df = df[df["text"].map(_is_ascii_major)]
    return df


In [ ]:
# SECTION 5 — Compute Features (sentiment, style, month)

def compute_features(df: pd.DataFrame, alpha: float=0.7) -> pd.DataFrame:
    df = df.copy()
    df["sent_text"]  = df["text"].map(_safe_vader_compound).astype(float)
    df["sent_stars"] = df["stars"].map(_stars_to_unit).astype(float)
    df["sent_hybrid"] = alpha*df["sent_text"] + (1.0-alpha)*df["sent_stars"]

    feats = df["text"].map(_style_feats).tolist()
    df[["style_exclam_rate","style_firstperson_rate","style_caps_rate"]] = pd.DataFrame(
        feats, index=df.index
    )
    df["month"] = df["ts"].map(_month_floor)
    return df


In [ ]:
# SECTION 6 — Aggregate to Per-User Trajectories + Drift Stats

def build_user_trajectories(df: pd.DataFrame, min_reviews: int=5):
    df = df.sort_values(["user_id","ts"]).copy()

    # keep users with >= min_reviews
    vc = df["user_id"].value_counts()
    keep = set(vc[vc >= min_reviews].index)
    df = df[df["user_id"].isin(keep)].copy()

    # optional within-user normalization for text sentiment
    df["text_z_user"] = df.groupby("user_id")["sent_text"].transform(
        lambda s: (s - s.mean()) / (s.std(ddof=0) + 1e-8)
    )

    # monthly aggregates
    monthly = (df.groupby(["user_id","month"])
                 .agg(sent_text_mean=("sent_text","mean"),
                      sent_hybrid_mean=("sent_hybrid","mean"),
                      stars_mean=("stars","mean"),
                      n_reviews=("text","count"))
                 .reset_index())

    # Use TRUE time for slope: month ordinal (YYYY*12 + MM)
    monthly["month_ord"] = monthly["month"].dt.year * 12 + monthly["month"].dt.month

    # slope + delta per user (on true time)
    def _slope_delta(g: pd.DataFrame) -> Dict[str,float]:
        g = g.sort_values("month_ord")
        x = g["month_ord"].values.astype(float)
        y = g["sent_hybrid_mean"].values.astype(float)
        if len(x) < 2 or np.isnan(y).all():
            return {"drift_slope": np.nan, "drift_delta": np.nan}
        xc = x - x.mean()
        denom = float(np.dot(xc, xc) + 1e-8)
        slope = float(np.dot(xc, (y - y.mean())) / denom)
        delta = float(y[-1] - y[0])
        return {"drift_slope": slope, "drift_delta": delta}

    drift = (monthly.groupby("user_id")
                  .apply(_slope_delta)
                  .apply(pd.Series)
                  .reset_index())

    #  total variation, sign-flip rate
    def _extra_metrics(g: pd.DataFrame) -> Dict[str, float]:
        y = g.sort_values("month_ord")["sent_hybrid_mean"].values.astype(float)
        if len(y) < 2 or np.isnan(y).all():
            return {"tv": np.nan, "flip_rate": np.nan}
        tv = float(np.nansum(np.abs(np.diff(y))))
        flips = float(np.nansum(np.sign(y[1:]) != np.sign(y[:-1])))
        flip_rate = flips / (len(y) - 1)
        return {"tv": tv, "flip_rate": flip_rate}

    extra = (monthly.groupby("user_id")
                   .apply(_extra_metrics)
                   .apply(pd.Series)
                   .reset_index())

    seqs = (monthly.groupby("user_id")
                  .agg(months=("month", list),
                       sent_hybrid_seq=("sent_hybrid_mean", list),
                       stars_seq=("stars_mean", list),
                       counts_seq=("n_reviews", list))
                  .reset_index())

    traj = pd.merge(drift, seqs, on="user_id", how="left")
    traj = pd.merge(traj, extra, on="user_id", how="left")
    return df, monthly, traj


In [ ]:
# SECTION 7 — Main: Config, Run, Save, Quick Plots

if __name__ == "__main__":
    INPUT = r"C:\Users\shubh\OneDrive\Desktop\nlp\Electronics_5.json\Electronics_5.json"   
    OUTDIR = "data"
    MIN_REVIEWS = 5
    ALPHA = 0.7
    N_LIMIT = 200000                

    os.makedirs(OUTDIR, exist_ok=True)

    print("[sanity] VADER available:", _VADER is not None)
    print("[sanity] 'I love it' →", _safe_vader_compound("I love it"))
    print("[sanity] 'I hate it' →", _safe_vader_compound("I hate it"))

    print("[load] reading:", INPUT)
    df = build_reviews_df(INPUT, n_limit=N_LIMIT)
    print(f"[rows] {len(df):,} reviews after basic cleaning")

    df = compute_features(df, alpha=ALPHA)
    print("[features] computed hybrid sentiment + style proxies + month bins")

    df_clean, monthly, traj = build_user_trajectories(df, min_reviews=MIN_REVIEWS)
    print(f"[users] {traj.shape[0]:,} users with ≥ {MIN_REVIEWS} reviews")

    df_clean.to_parquet(os.path.join(OUTDIR, "reviews.parquet"))
    monthly.to_parquet(os.path.join(OUTDIR, "reviews_monthly.parquet"))
    traj.to_parquet(os.path.join(OUTDIR, "user_trajectories.parquet"))
    traj.head(1000).to_csv(os.path.join(OUTDIR, "user_trajectories_preview.csv"), index=False)

    display_heads = (df_clean.head(), monthly.head(), traj.head())
    print(display_heads)

    n_samples = min(6, len(traj))
    if n_samples > 0:
        sample_users = traj.dropna(subset=["sent_hybrid_seq"]).sample(n_samples, random_state=42)["user_id"].tolist()
        for uid in sample_users:
            g = monthly[monthly["user_id"] == uid].sort_values("month")
            plt.figure()
            plt.plot(g["month"].dt.to_pydatetime(), g["sent_hybrid_mean"])
            plt.title(f"user {uid} — monthly hybrid sentiment")
            plt.xlabel("month")
            plt.ylabel("hybrid sentiment ([-1,1])")
            plt.tight_layout()
            plt.show()


[sanity] VADER available: True
[sanity] 'I love it' → 0.6369
[sanity] 'I hate it' → -0.5719
[load] reading: C:\Users\shubh\OneDrive\Desktop\nlp\Electronics_5.json\Electronics_5.json
[rows] 200,000 reviews after basic cleaning


KeyboardInterrupt: 

In [ ]:
n_samples = min(6, len(traj))
rep_users = traj.dropna(subset=["sent_hybrid_seq"]).sample(n_samples, random_state=42)["user_id"].tolist()

os.makedirs("figs", exist_ok=True)

for i, uid in enumerate(rep_users, start=1):
    g = monthly[monthly["user_id"] == uid].sort_values("month")
    plt.figure()
    # FutureWarning-safe: use .values for x
    plt.plot(g["month"].values, g["sent_hybrid_mean"])
    plt.title(f"user {uid} — monthly hybrid sentiment")
    plt.xlabel("month")
    plt.ylabel("hybrid sentiment ([-1,1])")
    plt.tight_layout()
    plt.savefig(f"figs/user_traj_{i}.png", dpi=220, bbox_inches="tight")
    plt.close()


In [ ]:
os.makedirs("tables", exist_ok=True)
cols = ["user_id", "drift_slope", "drift_delta", "tv", "flip_rate"]
head = pd.concat([
    traj.sort_values("drift_slope", ascending=False).head(5),
    traj.sort_values("drift_slope", ascending=True).head(5)
])[cols].drop_duplicates().head(10)

head = head.assign(
    drift_slope=head["drift_slope"].round(3),
    drift_delta=head["drift_delta"].round(3),
    tv=head["tv"].round(3),
    flip_rate=head["flip_rate"].round(3),
)

head.to_latex("tables/user_metrics.tex", index=False, escape=False)


In [ ]:
"""Baseline pipeline for ReviewMirror
"""

# SECTION 0 — Imports & setup

import os, gzip, json, re, datetime as dt
from typing import Iterator, Dict, Any, List, Tuple, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Sentiment (VADER)
try:
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    _VADER = SentimentIntensityAnalyzer()
except Exception:
    _VADER = None

_WORD = re.compile(r"\b\w+\b")
_FIRST_PERSON = re.compile(r"\b(i|i'm|i’ve|i've|my|me)\b", re.IGNORECASE)


# SECTION 1 — Reader for Amazon/Yelp formats

def _open_text(path: str):
    if path.endswith(".gz"):
        return gzip.open(path, "rt", encoding="utf-8", errors="ignore")
    return open(path, "rt", encoding="utf-8", errors="ignore")

def parse_amz_reviews(path: str, n_limit: Optional[int] = None) -> Iterator[Dict[str, Any]]:
    """
    Yields dict reviews from:
      - JSON Lines (usual Amazon format): one JSON object per line
      - OR a single large JSON array of objects
    """
    with _open_text(path) as f:
        head = f.read(2048)
        if not head:
            return
        is_array = head.lstrip().startswith("[")

    if is_array:
        with _open_text(path) as f:
            data = json.load(f)
            for i, obj in enumerate(data):
                yield obj
                if n_limit is not None and i + 1 >= n_limit:
                    break
    else:
        with _open_text(path) as f:
            for i, line in enumerate(f):
                line = line.strip()
                if not line:
                    continue
                try:
                    obj = json.loads(line)
                except Exception:
                    continue
                yield obj
                if n_limit is not None and i + 1 >= n_limit:
                    break


# SECTION 2 — Feature helpers (VADER, mapping, style, utils)

def _safe_vader_compound(text: str) -> float:
    """
    VADER compound in [-1,1]. Uses a global analyzer if available, else a simple fallback.
    """
    t = text or ""
    if _VADER is not None:
        try:
            return float(_VADER.polarity_scores(t)["compound"])
        except Exception:
            pass
    tl = t.lower()
    pos = sum(tl.count(w) for w in [" good", " great", " excellent", " amazing", " love", " perfect", " happy"])
    neg = sum(tl.count(w) for w in [" bad", " terrible", " awful", " hate", " poor", " broken", " angry"])
    return 0.0 if (pos + neg) == 0 else (pos - neg) / (pos + neg)

def _stars_to_unit(stars: float) -> float:
    """Map 1..5 -> [-1, 1]"""
    try:
        s = float(stars)
        return (s - 3.0) / 2.0
    except Exception:
        return np.nan

def _month_floor(ts: pd.Timestamp) -> pd.Timestamp:
    return pd.Timestamp(year=ts.year, month=ts.month, day=1, tz="UTC")

def _is_ascii_major(text) -> bool:
    if not isinstance(text, str):
        return True
    if not text:
        return True
    ascii_chars = sum(1 for ch in text if ord(ch) < 128)
    return ascii_chars >= 0.9 * len(text)

def _to_int_or_nan(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return np.nan
    if isinstance(x, str):
        x = x.replace(",", "").strip()
    try:
        return int(x)
    except Exception:
        return np.nan

def _style_feats(txt: str) -> Tuple[float, float, float]:
    """(exclam_per_char, first_person_per_token, caps_per_char)"""
    if not isinstance(txt, str) or not txt:
        return (0.0, 0.0, 0.0)
    n_chars = max(1, len(txt))
    tokens = _WORD.findall(txt)
    n_tok = max(1, len(tokens))
    exclam = txt.count("!") / n_chars
    first_person = len(_FIRST_PERSON.findall(txt)) / n_tok
    caps = sum(1 for ch in txt if ch.isupper()) / n_chars
    return exclam, first_person, caps


# SECTION 3 — Build row-level DataFrame

def build_reviews_df(path: str, n_limit: Optional[int] = None) -> pd.DataFrame:
    rows = []
    for rec in parse_amz_reviews(path, n_limit=n_limit):
        rows.append({
            "user_id": rec.get("reviewerID") or rec.get("user_id"),
            "item_id": rec.get("asin") or rec.get("business_id"),
            "text": rec.get("reviewText") or rec.get("text"),
            "stars": rec.get("overall") or rec.get("stars"),
            "ts": pd.to_datetime(
                rec.get("unixReviewTime"), unit="s", utc=True
            ) if rec.get("unixReviewTime") is not None else (
                pd.to_datetime(rec.get("reviewTime"), utc=True, errors="coerce")
                if rec.get("reviewTime") is not None else
                pd.to_datetime(rec.get("date"), utc=True, errors="coerce")
            ),
            "summary": rec.get("summary"),
            "helpful_votes": _to_int_or_nan(rec.get("vote") or rec.get("useful")),
        })
    df = pd.DataFrame(rows)
    # Clean
    df = df.dropna(subset=["user_id", "item_id", "ts"]).reset_index(drop=True)
    df = df[df["text"].map(_is_ascii_major)]
    return df


# SECTION 4 — Compute features (sentiment, style, month)

def compute_features(df: pd.DataFrame, alpha: float = 0.7) -> pd.DataFrame:
    df = df.copy()
    df["sent_text"]  = df["text"].map(_safe_vader_compound).astype(float)
    df["sent_stars"] = df["stars"].map(_stars_to_unit).astype(float)
    df["sent_hybrid"] = alpha * df["sent_text"] + (1.0 - alpha) * df["sent_stars"]

    feats = df["text"].map(_style_feats).tolist()
    df[["style_exclam_rate","style_firstperson_rate","style_caps_rate"]] = pd.DataFrame(
        feats, index=df.index
    )
    df["month"] = df["ts"].map(_month_floor)
    return df


# SECTION 5 — Aggregate to per-user trajectories + drift stats

def _group_apply_include_groups(grouper, func):
    try:
        return grouper.apply(func, include_groups=False)
    except TypeError:
        return grouper.apply(func)

def build_user_trajectories(df: pd.DataFrame, min_reviews: int = 5):
    df = df.sort_values(["user_id","ts"]).copy()

    # keep users with >= min_reviews
    vc = df["user_id"].value_counts()
    keep = set(vc[vc >= min_reviews].index)
    df = df[df["user_id"].isin(keep)].copy()

    df["text_z_user"] = df.groupby("user_id")["sent_text"].transform(
        lambda s: (s - s.mean()) / (s.std(ddof=0) + 1e-8)
    )

    # monthly aggregates
    monthly = (df.groupby(["user_id","month"])
                 .agg(sent_text_mean=("sent_text","mean"),
                      sent_hybrid_mean=("sent_hybrid","mean"),
                      stars_mean=("stars","mean"),
                      n_reviews=("text","count"))
                 .reset_index())

    # TRUE time for slope: month ordinal (YYYY*12 + MM)
    monthly["month_ord"] = monthly["month"].dt.year * 12 + monthly["month"].dt.month

    # slope + delta per user (on true time)
    def _slope_delta(g: pd.DataFrame) -> Dict[str, float]:
        g = g.sort_values("month_ord")
        x = g["month_ord"].values.astype(float)
        y = g["sent_hybrid_mean"].values.astype(float)
        if len(x) < 2 or np.isnan(y).all():
            return {"drift_slope": np.nan, "drift_delta": np.nan}
        xc = x - x.mean()
        denom = float(np.dot(xc, xc) + 1e-8)
        slope = float(np.dot(xc, (y - y.mean())) / denom)
        delta = float(y[-1] - y[0])
        return {"drift_slope": slope, "drift_delta": delta}

    drift = (_group_apply_include_groups(monthly.groupby("user_id"), _slope_delta)
                .apply(pd.Series)
                .reset_index())

    def _extra_metrics(g: pd.DataFrame) -> Dict[str, float]:
        y = g.sort_values("month_ord")["sent_hybrid_mean"].values.astype(float)
        if len(y) < 2 or np.isnan(y).all():
            return {"tv": np.nan, "flip_rate": np.nan}
        tv = float(np.nansum(np.abs(np.diff(y))))
        flips = float(np.nansum(np.sign(y[1:]) != np.sign(y[:-1])))
        flip_rate = flips / (len(y) - 1)
        return {"tv": tv, "flip_rate": flip_rate}

    extra = (_group_apply_include_groups(monthly.groupby("user_id"), _extra_metrics)
                .apply(pd.Series)
                .reset_index())

    seqs = (monthly.groupby("user_id")
                  .agg(months=("month", list),
                       sent_hybrid_seq=("sent_hybrid_mean", list),
                       stars_seq=("stars_mean", list),
                       counts_seq=("n_reviews", list))
                  .reset_index())

    traj = pd.merge(drift, seqs, on="user_id", how="left")
    traj = pd.merge(traj, extra, on="user_id", how="left")
    return df, monthly, traj


# SECTION 6 — Main: Config, run, save, plots

if __name__ == "__main__":
    CONFIG = {
        "dataset_name": "Amazon Electronics 5-core",
        "input_path": r"C:\Users\shubh\OneDrive\Desktop\nlp\Electronics_5.json\Electronics_5.json",
        "out_root": "runs",
        "run_name": "baseline_v1",
        "N_LIMIT": 200000,     
        "MIN_REVIEWS": 5,
        "ALPHA": 0.7,
        "seed": 42
    }
    np.random.seed(CONFIG["seed"])

    run_id = f'{CONFIG["run_name"]}_{dt.datetime.now().strftime("%Y%m%d_%H%M%S")}'
    RUN_DIR = os.path.join(CONFIG["out_root"], run_id)
    FIG_DIR = os.path.join(RUN_DIR, "figs")
    DATA_DIR = os.path.join(RUN_DIR, "data")
    os.makedirs(FIG_DIR, exist_ok=True)
    os.makedirs(DATA_DIR, exist_ok=True)
    print(f"[run] created run directory: {RUN_DIR}")

    with open(os.path.join(RUN_DIR, "config.json"), "w", encoding="utf-8") as f:
        json.dump(CONFIG, f, indent=2)

    # 3) Load + preprocess
    INPUT = CONFIG["input_path"]
    N_LIMIT = CONFIG["N_LIMIT"]
    MIN_REVIEWS = CONFIG["MIN_REVIEWS"]
    ALPHA = CONFIG["ALPHA"]

    print("[sanity] VADER available:", _VADER is not None)
    print("[sanity] 'I love it' →", _safe_vader_compound("I love it"))
    print("[sanity] 'I hate it' →", _safe_vader_compound("I hate it"))

    print("[sanity] baseline config:", CONFIG)
    print("[load] reading:", INPUT)
    df = build_reviews_df(INPUT, n_limit=N_LIMIT)
    print(f"[rows] {len(df):,} reviews after cleaning")

    df = compute_features(df, alpha=ALPHA)
    print("[features] computed hybrid sentiment + style proxies + month bins")

    df_clean, monthly, traj = build_user_trajectories(df, min_reviews=MIN_REVIEWS)
    print(f"[users] {traj.shape[0]:,} users with ≥ {MIN_REVIEWS} reviews")

    df_clean.to_parquet(os.path.join(DATA_DIR, "reviews.parquet"))
    monthly.to_parquet(os.path.join(DATA_DIR, "reviews_monthly.parquet"))
    traj.to_parquet(os.path.join(DATA_DIR, "user_trajectories.parquet"))
    traj.head(1000).to_csv(os.path.join(DATA_DIR, "user_trajectories_preview.csv"), index=False)
    print("[save] wrote parquet + preview to", DATA_DIR)

    metrics = {
        "n_rows": int(len(df_clean)),
        "n_users": int(traj["user_id"].nunique()),
        "n_items": int(df_clean["item_id"].nunique()),
        "min_reviews_per_user": int(MIN_REVIEWS),
        "alpha_hybrid": float(ALPHA),
        "stars_mean": float(df_clean["stars"].mean()),
        "stars_std": float(df_clean["stars"].std()),
        "sent_text_mean": float(df_clean["sent_text"].mean()),
        "sent_text_std": float(df_clean["sent_text"].std()),
        "sent_hybrid_mean": float(df_clean["sent_hybrid"].mean()),
        "sent_hybrid_std": float(df_clean["sent_hybrid"].std()),
    }
    for col in ["drift_slope", "drift_delta", "tv", "flip_rate"]:
        if col in traj.columns:
            metrics[f"{col}_mean"] = float(traj[col].mean())
            metrics[f"{col}_std"] = float(traj[col].std())
            metrics[f"{col}_min"] = float(traj[col].min())
            metrics[f"{col}_max"] = float(traj[col].max())

    with open(os.path.join(RUN_DIR, "metrics_baseline.json"), "w", encoding="utf-8") as f:
        json.dump(metrics, f, indent=2)
    print("[metrics] saved summary metrics to metrics_baseline.json")

    n_samples = min(6, len(traj))
    if n_samples > 0:
        sample_users = traj.dropna(subset=["sent_hybrid_seq"]).sample(
            n_samples, random_state=CONFIG["seed"]
        )["user_id"].tolist()

        for i, uid in enumerate(sample_users, start=1):
            g = monthly[monthly["user_id"] == uid].sort_values("month")
            plt.figure()
            # FutureWarning-safe: np.array(...)
            plt.plot(np.array(g["month"].values), g["sent_hybrid_mean"])
            plt.title(f"user {uid} — monthly hybrid sentiment")
            plt.xlabel("month")
            plt.ylabel("hybrid sentiment ([-1,1])")
            plt.tight_layout()
            fig_path = os.path.join(FIG_DIR, f"user_traj_{i}.png")
            plt.savefig(fig_path, dpi=220, bbox_inches="tight")
            plt.close()
            print(f"[fig] saved {fig_path}")

    print("[done] baseline run complete.")


[run] created run directory: runs\baseline_v1_20251110_183220
[sanity] VADER available: True
[sanity] 'I love it' → 0.6369
[sanity] 'I hate it' → -0.5719
[sanity] baseline config: {'dataset_name': 'Amazon Electronics 5-core', 'input_path': 'C:\\Users\\shubh\\OneDrive\\Desktop\\nlp\\Electronics_5.json\\Electronics_5.json', 'out_root': 'runs', 'run_name': 'baseline_v1', 'N_LIMIT': 200000, 'MIN_REVIEWS': 5, 'ALPHA': 0.7, 'seed': 42}
[load] reading: C:\Users\shubh\OneDrive\Desktop\nlp\Electronics_5.json\Electronics_5.json
[rows] 200,000 reviews after cleaning
[features] computed hybrid sentiment + style proxies + month bins
[users] 2,657 users with ≥ 5 reviews
[save] wrote parquet + preview to runs\baseline_v1_20251110_183220\data
[metrics] saved summary metrics to metrics_baseline.json
[fig] saved runs\baseline_v1_20251110_183220\figs\user_traj_1.png
[fig] saved runs\baseline_v1_20251110_183220\figs\user_traj_2.png
[fig] saved runs\baseline_v1_20251110_183220\figs\user_traj_3.png
[fig] sa

new

In [ ]:
# SECTION 7 — Data Splits for Evaluation

def create_data_splits(traj: pd.DataFrame, monthly: pd.DataFrame, 
                       split_ratio=(0.7, 0.15, 0.15), strategy="temporal"):
    """
    Split users into train/val/test sets for reproducible evaluation.
    
    Args:
        traj: User trajectories dataframe
        monthly: Monthly aggregated reviews
        split_ratio: (train, val, test) proportions
        strategy: 'temporal' or 'random'
    
    Returns:
        Dict with 'train', 'val', 'test' user lists
    """
    if strategy == "temporal":
        user_first_review = monthly.groupby("user_id")["month"].min()
        sorted_users = user_first_review.sort_values().index.tolist()
    else:
        sorted_users = traj["user_id"].sample(frac=1, random_state=42).tolist()
    
    n = len(sorted_users)
    train_end = int(n * split_ratio[0])
    val_end = train_end + int(n * split_ratio[1])
    
    return {
        "train": sorted_users[:train_end],
        "val": sorted_users[train_end:val_end],
        "test": sorted_users[val_end:]
    }


# SECTION 8 — Baseline Metrics & Evaluation

def compute_baseline_statistics(traj: pd.DataFrame, splits: Dict):
    """
    Compute baseline statistics from training set.
    These serve as naive predictors (e.g., "predict mean drift for all users").
    """
    train_traj = traj[traj["user_id"].isin(splits["train"])]
    
    baselines = {
        "mean_drift_slope": float(train_traj["drift_slope"].mean()),
        "mean_drift_delta": float(train_traj["drift_delta"].mean()),
        "median_drift_slope": float(train_traj["drift_slope"].median()),
        "median_drift_delta": float(train_traj["drift_delta"].median()),
        "std_drift_slope": float(train_traj["drift_slope"].std()),
        "std_drift_delta": float(train_traj["drift_delta"].std()),
    }
    
    return baselines


def evaluate_drift_metrics(traj: pd.DataFrame, splits: Dict, baselines: Dict):
    """
    Evaluate drift detection on test set.
    Reports distribution stats + how far users deviate from baseline assumptions.
    """
    test_traj = traj[traj["user_id"].isin(splits["test"])]
    
    metrics = {
        "test_size": int(len(test_traj)),
        
        "slope_mae": float(np.abs(test_traj["drift_slope"] - baselines["mean_drift_slope"]).mean()),
        "delta_mae": float(np.abs(test_traj["drift_delta"] - baselines["mean_drift_delta"]).mean()),
        
        "high_drift_users": int((np.abs(test_traj["drift_slope"]) > 0.01).sum()),
        "volatile_users": int((test_traj["tv"] > test_traj["tv"].quantile(0.75)).sum()),
        "flat_users": int((np.abs(test_traj["drift_slope"]) < 0.001).sum()),
    }
    
    for col in ["drift_slope", "drift_delta", "tv", "flip_rate"]:
        if col in test_traj.columns:
            metrics[f"test_{col}_mean"] = float(test_traj[col].mean())
            metrics[f"test_{col}_median"] = float(test_traj[col].median())
            metrics[f"test_{col}_std"] = float(test_traj[col].std())
            metrics[f"test_{col}_q25"] = float(test_traj[col].quantile(0.25))
            metrics[f"test_{col}_q75"] = float(test_traj[col].quantile(0.75))
    
    return metrics


# SECTION 9 — Enhanced Visualizations

def create_diagnostic_plots(traj: pd.DataFrame, monthly: pd.DataFrame, fig_dir: str):
    
    # 1. Distribution of drift metrics (4-panel)
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    drift_cols = ["drift_slope", "drift_delta", "tv", "flip_rate"]
    
    for ax, col in zip(axes.flat, drift_cols):
        data = traj[col].dropna()
        ax.hist(data, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
        ax.axvline(data.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {data.mean():.3f}')
        ax.axvline(data.median(), color='orange', linestyle='--', linewidth=2, label=f'Median: {data.median():.3f}')
        ax.set_title(f"Distribution of {col}", fontsize=12, fontweight='bold')
        ax.set_xlabel(col)
        ax.set_ylabel("Count")
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, "drift_distributions.png"), dpi=220)
    plt.close()
    print(f"[fig] saved {os.path.join(fig_dir, 'drift_distributions.png')}")
    
    monthly_agg = monthly.groupby("month").agg({
        "sent_hybrid_mean": "mean",
        "stars_mean": "mean",
        "n_reviews": "sum"
    }).reset_index()
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    ax1.plot(monthly_agg["month"], monthly_agg["sent_hybrid_mean"], 
             marker='o', linewidth=2, markersize=4, color='darkgreen')
    ax1.axhline(0, color='gray', linestyle='--', alpha=0.5)
    ax1.set_title("Average Hybrid Sentiment Over Time (All Users)", fontsize=14, fontweight='bold')
    ax1.set_ylabel("Mean Sentiment", fontsize=11)
    ax1.grid(True, alpha=0.3)
    
    ax2.bar(monthly_agg["month"], monthly_agg["n_reviews"], alpha=0.7, color='coral')
    ax2.set_title("Review Volume Over Time", fontsize=14, fontweight='bold')
    ax2.set_xlabel("Month", fontsize=11)
    ax2.set_ylabel("Number of Reviews", fontsize=11)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, "temporal_trends.png"), dpi=220)
    plt.close()
    print(f"[fig] saved {os.path.join(fig_dir, 'temporal_trends.png')}")
    plt.figure(figsize=(10, 8))
    plt.scatter(traj["drift_slope"], traj["tv"], alpha=0.4, s=20)
    plt.axvline(0, color='red', linestyle='--', alpha=0.5)
    plt.axhline(traj["tv"].median(), color='orange', linestyle='--', alpha=0.5, 
                label=f'Median TV: {traj["tv"].median():.2f}')
    plt.xlabel("Drift Slope (sentiment change rate)", fontsize=11)
    plt.ylabel("Total Variation (volatility)", fontsize=11)
    plt.title("Relationship: Drift Slope vs. Volatility", fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, "slope_vs_volatility.png"), dpi=220)
    plt.close()
    print(f"[fig] saved {os.path.join(fig_dir, 'slope_vs_volatility.png')}")


# SECTION 10 — Reproducibility Utilities

def create_reproducibility_manifest(run_dir: str, config: dict, df: pd.DataFrame):
    """Document all information needed to reproduce this run."""
    import sys
    
    manifest = {
        "run_info": {
            "run_id": os.path.basename(run_dir),
            "timestamp": dt.datetime.now().isoformat(),
            "python_version": sys.version,
            "package_versions": {
                "pandas": pd.__version__,
                "numpy": np.__version__,
                "matplotlib": plt.matplotlib.__version__,
            }
        },
        "config": config,
        "data_summary": {
            "total_reviews": int(len(df)),
            "date_range": {
                "min": str(df["ts"].min()),
                "max": str(df["ts"].max()),
                "span_days": int((df["ts"].max() - df["ts"].min()).days)
            },
            "unique_users": int(df["user_id"].nunique()),
            "unique_items": int(df["item_id"].nunique()),
        },
        "preprocessing_pipeline": [
            "1. Load JSON/JSONL reviews (gzip-compatible)",
            "2. Parse reviewerID/asin, reviewText, overall, unixReviewTime",
            "3. Filter: drop rows with missing user_id, item_id, or timestamp",
            "4. Filter: keep only reviews with >=90% ASCII characters",
            "5. Compute VADER sentiment on review text → sent_text",
            "6. Normalize star ratings (1-5) to [-1,+1] → sent_stars",
            f"7. Create hybrid sentiment: {config['ALPHA']}*sent_text + {1-config['ALPHA']}*sent_stars",
            "8. Extract style features: exclamation rate, first-person rate, caps rate",
            "9. Aggregate to monthly bins per user",
            f"10. Keep only users with >={config['MIN_REVIEWS']} reviews",
            "11. Compute drift metrics: slope (linear), delta (first-to-last), TV, flip_rate"
        ]
    }
    
    with open(os.path.join(run_dir, "manifest.json"), "w", encoding="utf-8") as f:
        json.dump(manifest, f, indent=2)
    print(f"[manifest] saved to {os.path.join(run_dir, 'manifest.json')}")


def create_run_readme(run_dir: str, config: dict, metrics: dict):
    readme = f"""# ReviewMirror Baseline Run

**Run ID**: `{os.path.basename(run_dir)}`  
**Date**: {dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

---

## 📊 Configuration

| Parameter | Value |
|-----------|-------|
| Dataset | {config['dataset_name']} |
| Reviews Analyzed | {config['N_LIMIT'] if config['N_LIMIT'] else 'ALL'} |
| Min Reviews/User | {config['MIN_REVIEWS']} |
| Hybrid Alpha (text weight) | {config['ALPHA']} |
| Random Seed | {config['seed']} |

---

## 📈 Data Summary

- **Total reviews** (after cleaning): {metrics.get('n_rows', 'N/A'):,}
- **Unique users**: {metrics.get('n_users', 'N/A'):,}
- **Unique items**: {metrics.get('n_items', 'N/A'):,}
- **Average stars**: {metrics.get('stars_mean', 0):.2f} ± {metrics.get('stars_std', 0):.2f}

---

## 🔍 Key Findings

### Drift Metrics (All Users)
| Metric | Mean | Std | Min | Max |
|--------|------|-----|-----|-----|
| Drift Slope | {metrics.get('drift_slope_mean', 0):.4f} | {metrics.get('drift_slope_std', 0):.4f} | {metrics.get('drift_slope_min', 0):.4f} | {metrics.get('drift_slope_max', 0):.4f} |
| Drift Delta | {metrics.get('drift_delta_mean', 0):.4f} | {metrics.get('drift_delta_std', 0):.4f} | {metrics.get('drift_delta_min', 0):.4f} | {metrics.get('drift_delta_max', 0):.4f} |
| Total Variation | {metrics.get('tv_mean', 0):.4f} | {metrics.get('tv_std', 0):.4f} | {metrics.get('tv_min', 0):.4f} | {metrics.get('tv_max', 0):.4f} |

### Interesting User Counts
- **High-drift users** (|slope| > 0.01): {metrics.get('high_drift_users', 'N/A')}
- **Volatile users** (TV > 75th percentile): {metrics.get('volatile_users', 'N/A')}
- **Flat users** (|slope| < 0.001): {metrics.get('flat_users', 'N/A')}

---

## 📁 Output Files
```
{os.path.basename(run_dir)}/
├── data/
│   ├── reviews.parquet              # Review-level data with features
│   ├── reviews_monthly.parquet      # Monthly aggregates per user
│   ├── user_trajectories.parquet    # Per-user drift metrics
│   ├── splits.json                  # Train/val/test split
│   └── user_trajectories_preview.csv
├── figs/
│   ├── drift_distributions.png      # Histograms of drift metrics
│   ├── temporal_trends.png          # Sentiment over time
│   ├── slope_vs_volatility.png      # Correlation plot
│   └── user_traj_*.png              # Sample user trajectories
├── config.json                       # Full configuration
├── metrics_baseline.json             # Quantitative metrics
├── manifest.json                     # Reproducibility info
└── README.md                         # This file
```

---

## 🧠 Modeling Rationale

### Why Hybrid Sentiment?
Combines two complementary signals:
- **Text sentiment** (VADER): Captures nuanced language (e.g., "good but...")
- **Star ratings**: Captures user's explicit judgment

Hybrid formula: `{config['ALPHA']} × text + {1-config['ALPHA']} × stars`

### Why These Drift Metrics?

1. **Drift Slope**: Linear trend coefficient
   - Robust to noise via least-squares fitting
   - Interpretable: "sentiment changes by X per month"

2. **Drift Delta**: First − Last sentiment
   - Captures endpoint difference
   - Complementary to slope (non-linear changes)

3. **Total Variation (TV)**: Sum of absolute month-to-month changes
   - Measures volatility/instability
   - Independent of overall direction

4. **Flip Rate**: Proportion of sign changes
   - Detects erratic behavior
   - Normalized for trajectory length

---

## 🚀 Next Steps

- [ ] Experiment with different `ALPHA` values (text vs. stars weight)
- [ ] Try transformer-based sentiment (e.g., fine-tuned BERT)
- [ ] Cluster users by drift pattern (k-means on slope/TV)
- [ ] Build predictive model: forecast future drift from early reviews
- [ ] Analyze item-level effects (do certain products induce drift?)

---

## 📝 Notes

This is a **statistical baseline** for ReviewMirror. No machine learning models trained yet.
Goal: Establish ground truth about drift patterns in the data before building predictive systems.
"""
    
    with open(os.path.join(run_dir, "README.md"), "w", encoding="utf-8") as f:
        f.write(readme)
    print(f"[readme] saved to {os.path.join(run_dir, 'README.md')}")


# SECTION 11 — Sanity Checks

def run_sanity_checks(df: pd.DataFrame, traj: pd.DataFrame):
    issues = []
    
    nan_frac = df["sent_hybrid"].isna().sum() / len(df)
    if nan_frac > 0.1:
        issues.append(f"WARNING: {nan_frac:.1%} NaN in hybrid sentiment")
    
    date_span = (df["ts"].max() - df["ts"].min()).days
    if date_span < 30:
        issues.append(f"WARNING: Data spans only {date_span} days")
    
    max_slope = traj["drift_slope"].abs().max()
    if max_slope > 1.0:
        issues.append(f"WARNING: Extreme drift_slope detected: {max_slope:.3f}")
    
    max_reviews = df.groupby("user_id").size().max()
    median_reviews = df.groupby("user_id").size().median()
    if max_reviews > median_reviews * 20:
        issues.append(f"WARNING: Max reviews per user ({max_reviews}) >> median ({median_reviews})")
    
    return issues

In [ ]:
# SECTION 12 — Main Pipeline 

if __name__ == "__main__":
    CONFIG = {
        "dataset_name": "Amazon Electronics 5-core",
        "input_path": r"C:\Users\shubh\OneDrive\Desktop\nlp\Electronics_5.json\Electronics_5.json",
        "out_root": "runs",
        "run_name": "baseline_v1",
        "N_LIMIT": 200000,
        "MIN_REVIEWS": 5,
        "ALPHA": 0.7,
        "seed": 42,
        "split_strategy": "temporal",  # NEW
        "split_ratio": [0.7, 0.15, 0.15]  # NEW: train/val/test
    }
    np.random.seed(CONFIG["seed"])

    run_id = f'{CONFIG["run_name"]}_{dt.datetime.now().strftime("%Y%m%d_%H%M%S")}'
    RUN_DIR = os.path.join(CONFIG["out_root"], run_id)
    FIG_DIR = os.path.join(RUN_DIR, "figs")
    DATA_DIR = os.path.join(RUN_DIR, "data")
    os.makedirs(FIG_DIR, exist_ok=True)
    os.makedirs(DATA_DIR, exist_ok=True)
    print(f"[run] created run directory: {RUN_DIR}")

    with open(os.path.join(RUN_DIR, "config.json"), "w", encoding="utf-8") as f:
        json.dump(CONFIG, f, indent=2)

    INPUT = CONFIG["input_path"]
    N_LIMIT = CONFIG["N_LIMIT"]
    MIN_REVIEWS = CONFIG["MIN_REVIEWS"]
    ALPHA = CONFIG["ALPHA"]

    print("[sanity] VADER available:", _VADER is not None)
    print("[sanity] 'I love it' →", _safe_vader_compound("I love it"))
    print("[sanity] 'I hate it' →", _safe_vader_compound("I hate it"))
    print("[config]", CONFIG)

    print(f"[load] reading: {INPUT}")
    df = build_reviews_df(INPUT, n_limit=N_LIMIT)
    print(f"[rows] {len(df):,} reviews after cleaning")

    df = compute_features(df, alpha=ALPHA)
    print("[features] computed hybrid sentiment + style proxies + month bins")

    df_clean, monthly, traj = build_user_trajectories(df, min_reviews=MIN_REVIEWS)
    print(f"[users] {traj.shape[0]:,} users with ≥ {MIN_REVIEWS} reviews")

    issues = run_sanity_checks(df_clean, traj)
    for issue in issues:
        print(f"[check] {issue}")

    splits = create_data_splits(traj, monthly, 
                                split_ratio=CONFIG["split_ratio"], 
                                strategy=CONFIG["split_strategy"])
    print(f"[splits] Train: {len(splits['train'])}, Val: {len(splits['val'])}, Test: {len(splits['test'])}")
    
    with open(os.path.join(DATA_DIR, "splits.json"), "w", encoding="utf-8") as f:
        json.dump({k: list(v) for k, v in splits.items()}, f, indent=2)

    baselines = compute_baseline_statistics(traj, splits)
    print(f"[baseline] Mean drift slope: {baselines['mean_drift_slope']:.4f}")
    print(f"[baseline] Median drift slope: {baselines['median_drift_slope']:.4f}")
    
    with open(os.path.join(RUN_DIR, "baselines.json"), "w", encoding="utf-8") as f:
        json.dump(baselines, f, indent=2)

    df_clean.to_parquet(os.path.join(DATA_DIR, "reviews.parquet"))
    monthly.to_parquet(os.path.join(DATA_DIR, "reviews_monthly.parquet"))
    traj.to_parquet(os.path.join(DATA_DIR, "user_trajectories.parquet"))
    traj.head(1000).to_csv(os.path.join(DATA_DIR, "user_trajectories_preview.csv"), index=False)
    print("[save] wrote parquet + preview to", DATA_DIR)

    metrics = {
        "n_rows": int(len(df_clean)),
        "n_users": int(traj["user_id"].nunique()),
        "n_items": int(df_clean["item_id"].nunique()),
        "min_reviews_per_user": int(MIN_REVIEWS),
        "alpha_hybrid": float(ALPHA),
        "stars_mean": float(df_clean["stars"].mean()),
        "stars_std": float(df_clean["stars"].std()),
        "sent_text_mean": float(df_clean["sent_text"].mean()),
        "sent_text_std": float(df_clean["sent_text"].std()),
        "sent_hybrid_mean": float(df_clean["sent_hybrid"].mean()),
        "sent_hybrid_std": float(df_clean["sent_hybrid"].std()),
    }
    
    for col in ["drift_slope", "drift_delta", "tv", "flip_rate"]:
        if col in traj.columns:
            metrics[f"{col}_mean"] = float(traj[col].mean())
            metrics[f"{col}_std"] = float(traj[col].std())
            metrics[f"{col}_min"] = float(traj[col].min())
            metrics[f"{col}_max"] = float(traj[col].max())

    eval_metrics = evaluate_drift_metrics(traj, splits, baselines)
    metrics.update(eval_metrics)
    
    with open(os.path.join(RUN_DIR, "metrics_baseline.json"), "w", encoding="utf-8") as f:
        json.dump(metrics, f, indent=2)
    print("[metrics] saved summary + evaluation metrics")

    create_diagnostic_plots(traj, monthly, FIG_DIR)

    n_samples = min(6, len(traj))
    if n_samples > 0:
        sample_users = traj.dropna(subset=["sent_hybrid_seq"]).sample(
            n_samples, random_state=CONFIG["seed"]
        )["user_id"].tolist()

        for i, uid in enumerate(sample_users, start=1):
            g = monthly[monthly["user_id"] == uid].sort_values("month")
            plt.figure(figsize=(10, 6))
            plt.plot(np.array(g["month"].values), g["sent_hybrid_mean"], 
                     marker='o', linewidth=2, markersize=6)
            plt.axhline(0, color='gray', linestyle='--', alpha=0.5)
            plt.title(f"User {uid} — Monthly Hybrid Sentiment Trajectory", 
                     fontsize=13, fontweight='bold')
            plt.xlabel("Month", fontsize=11)
            plt.ylabel("Hybrid Sentiment ([-1, 1])", fontsize=11)
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            fig_path = os.path.join(FIG_DIR, f"user_traj_{i}.png")
            plt.savefig(fig_path, dpi=220, bbox_inches="tight")
            plt.close()
            print(f"[fig] saved {fig_path}")

    create_reproducibility_manifest(RUN_DIR, CONFIG, df_clean)
    create_run_readme(RUN_DIR, CONFIG, metrics)

    print(f"\n{'='*60}")
    print(f"[DONE] Baseline run complete!")
    print(f"[DONE] All outputs saved to: {RUN_DIR}")
    print(f"{'='*60}\n")




[run] created run directory: runs\baseline_v1_20251110_195108
[sanity] VADER available: True
[sanity] 'I love it' → 0.6369
[sanity] 'I hate it' → -0.5719
[config] {'dataset_name': 'Amazon Electronics 5-core', 'input_path': 'C:\\Users\\shubh\\OneDrive\\Desktop\\nlp\\Electronics_5.json\\Electronics_5.json', 'out_root': 'runs', 'run_name': 'baseline_v1', 'N_LIMIT': 200000, 'MIN_REVIEWS': 5, 'ALPHA': 0.7, 'seed': 42, 'split_strategy': 'temporal', 'split_ratio': [0.7, 0.15, 0.15]}
[load] reading: C:\Users\shubh\OneDrive\Desktop\nlp\Electronics_5.json\Electronics_5.json
[rows] 200,000 reviews after cleaning
[features] computed hybrid sentiment + style proxies + month bins


C:\Users\shubh\AppData\Local\Temp\ipykernel_19956\506542110.py:43: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_slope_delta)
C:\Users\shubh\AppData\Local\Temp\ipykernel_19956\506542110.py:58: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_extra_metrics)


[users] 2,657 users with ≥ 5 reviews
[check] WARNING: Extreme drift_slope detected: 1.275
[check] WARNING: Max reviews per user (149) >> median (6.0)
[splits] Train: 1859, Val: 398, Test: 400
[baseline] Mean drift slope: 0.0005
[baseline] Median drift slope: -0.0002
[save] wrote parquet + preview to runs\baseline_v1_20251110_195108\data
[metrics] saved summary + evaluation metrics
[fig] saved runs\baseline_v1_20251110_195108\figs\drift_distributions.png
[fig] saved runs\baseline_v1_20251110_195108\figs\temporal_trends.png
[fig] saved runs\baseline_v1_20251110_195108\figs\slope_vs_volatility.png
[fig] saved runs\baseline_v1_20251110_195108\figs\user_traj_1.png
[fig] saved runs\baseline_v1_20251110_195108\figs\user_traj_2.png
[fig] saved runs\baseline_v1_20251110_195108\figs\user_traj_3.png
[fig] saved runs\baseline_v1_20251110_195108\figs\user_traj_4.png
[fig] saved runs\baseline_v1_20251110_195108\figs\user_traj_5.png
[fig] saved runs\baseline_v1_20251110_195108\figs\user_traj_6.png
[m

neww

In [ ]:
def create_data_splits(traj: pd.DataFrame, monthly: pd.DataFrame, 
                       split_ratio=(0.7, 0.15, 0.15), strategy="temporal"):
    if strategy == "temporal":
        # Sort by first review date per user
        user_first_review = monthly.groupby("user_id")["month"].min()
        sorted_users = user_first_review.sort_values().index.tolist()
    else:
        sorted_users = traj["user_id"].sample(frac=1, random_state=42).tolist()
    
    n = len(sorted_users)
    train_end = int(n * split_ratio[0])
    val_end = train_end + int(n * split_ratio[1])
    
    return {
        "train": sorted_users[:train_end],
        "val": sorted_users[train_end:val_end],
        "test": sorted_users[val_end:]
    }

In [ ]:
def compute_baselines(traj: pd.DataFrame, splits: Dict):
    """
    Compute simple baseline predictions:
    - Random: random drift direction
    - Mean: dataset mean drift
    - Last-value: assume no change from last observation
    """
    train_traj = traj[traj["user_id"].isin(splits["train"])]
    
    baselines = {
        "mean_drift_slope": train_traj["drift_slope"].mean(),
        "mean_drift_delta": train_traj["drift_delta"].mean(),
        "std_drift_slope": train_traj["drift_slope"].std(),
        "std_drift_delta": train_traj["drift_delta"].std(),
    }
    
    # Save baselines
    return baselines

In [ ]:
def evaluate_drift_metrics(traj: pd.DataFrame, splits: Dict, baselines: Dict):
    """
    Evaluate drift detection performance on test set
    """
    test_traj = traj[traj["user_id"].isin(splits["test"])]
    
    metrics = {
        "test_size": len(test_traj),
        "slope_mae": np.abs(test_traj["drift_slope"] - baselines["mean_drift_slope"]).mean(),
        "delta_mae": np.abs(test_traj["drift_delta"] - baselines["mean_drift_delta"]).mean(),
        "slope_rmse": np.sqrt(((test_traj["drift_slope"] - baselines["mean_drift_slope"])**2).mean()),
        "high_drift_users": (np.abs(test_traj["drift_slope"]) > 0.01).sum(),
        "volatile_users": (test_traj["tv"] > test_traj["tv"].quantile(0.75)).sum(),
    }
    
    # Distribution statistics
    for col in ["drift_slope", "drift_delta", "tv", "flip_rate"]:
        metrics[f"test_{col}_mean"] = test_traj[col].mean()
        metrics[f"test_{col}_median"] = test_traj[col].median()
        metrics[f"test_{col}_q25"] = test_traj[col].quantile(0.25)
        metrics[f"test_{col}_q75"] = test_traj[col].quantile(0.75)
    
    return metrics

In [ ]:
import logging

def setup_logging(run_dir: str):
    """Setup logging to both file and console"""
    log_path = os.path.join(run_dir, "pipeline.log")
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_path),
            logging.StreamHandler()
        ]
    )
    return logging.getLogger(__name__)

# Then use: logger.info(f"[load] reading: {INPUT}")

In [ ]:
def create_diagnostic_plots(traj: pd.DataFrame, monthly: pd.DataFrame, fig_dir: str):
    
    # 1. Distribution of drift metrics
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    for ax, col in zip(axes.flat, ["drift_slope", "drift_delta", "tv", "flip_rate"]):
        ax.hist(traj[col].dropna(), bins=50, edgecolor='black', alpha=0.7)
        ax.set_title(f"Distribution of {col}")
        ax.set_xlabel(col)
        ax.set_ylabel("Count")
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, "drift_distributions.png"), dpi=220)
    plt.close()
    
    # 2. Correlation matrix
    corr_cols = ["drift_slope", "drift_delta", "tv", "flip_rate"]
    corr_matrix = traj[corr_cols].corr()
    plt.figure(figsize=(8, 6))
    plt.imshow(corr_matrix, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
    plt.colorbar(label='Correlation')
    plt.xticks(range(len(corr_cols)), corr_cols, rotation=45)
    plt.yticks(range(len(corr_cols)), corr_cols)
    for i in range(len(corr_cols)):
        for j in range(len(corr_cols)):
            plt.text(j, i, f'{corr_matrix.iloc[i, j]:.2f}', 
                    ha='center', va='center')
    plt.title("Correlation Matrix of Drift Metrics")
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, "correlation_matrix.png"), dpi=220)
    plt.close()
    
    # 3. Time series of monthly sentiment (aggregated)
    monthly_agg = monthly.groupby("month").agg({
        "sent_hybrid_mean": "mean",
        "stars_mean": "mean",
        "n_reviews": "sum"
    }).reset_index()
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    ax1.plot(monthly_agg["month"], monthly_agg["sent_hybrid_mean"], marker='o')
    ax1.set_title("Average Hybrid Sentiment Over Time")
    ax1.set_ylabel("Mean Sentiment")
    ax1.grid(True, alpha=0.3)
    
    ax2.bar(monthly_agg["month"], monthly_agg["n_reviews"], alpha=0.7)
    ax2.set_title("Review Volume Over Time")
    ax2.set_xlabel("Month")
    ax2.set_ylabel("Number of Reviews")
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, "temporal_trends.png"), dpi=220)
    plt.close()

In [ ]:
from scipy import stats

def statistical_tests(traj: pd.DataFrame):
    results = {}
    
    # Test if drift_slope significantly different from 0
    t_stat, p_val = stats.ttest_1samp(traj["drift_slope"].dropna(), 0)
    results["slope_vs_zero"] = {"t_statistic": float(t_stat), "p_value": float(p_val)}
    
    # Correlation between slope and volatility
    corr, p_val = stats.pearsonr(
        traj["drift_slope"].dropna(), 
        traj["tv"].dropna()
    )
    results["slope_tv_correlation"] = {"correlation": float(corr), "p_value": float(p_val)}
    
    return results

In [ ]:
def create_reproducibility_manifest(run_dir: str, config: dict, df: pd.DataFrame):
    """Document everything needed to reproduce the run"""
    manifest = {
        "run_info": {
            "run_id": os.path.basename(run_dir),
            "timestamp": dt.datetime.now().isoformat(),
            "python_version": sys.version,
            "package_versions": {
                "pandas": pd.__version__,
                "numpy": np.__version__,
                # Add other packages
            }
        },
        "config": config,
        "data_stats": {
            "total_reviews": len(df),
            "date_range": {
                "min": df["ts"].min().isoformat(),
                "max": df["ts"].max().isoformat()
            },
            "unique_users": df["user_id"].nunique(),
            "unique_items": df["item_id"].nunique(),
        },
        "preprocessing_steps": [
            "1. Load JSON/JSONL reviews",
            "2. Filter non-ASCII text (>=90% ASCII chars)",
            "3. Drop rows missing user_id, item_id, or timestamp",
            "4. Compute VADER sentiment + star-based sentiment",
            "5. Create hybrid sentiment (alpha*text + (1-alpha)*stars)",
            "6. Aggregate to monthly user-level trajectories",
            "7. Compute drift metrics (slope, delta, TV, flip_rate)"
        ]
    }
    
    with open(os.path.join(run_dir, "manifest.json"), "w") as f:
        json.dump(manifest, f, indent=2, default=str)

In [ ]:
def run_sanity_checks(df: pd.DataFrame, traj: pd.DataFrame):
    issues = []
    
    # Check for unexpected NaN patterns
    if df["sent_hybrid"].isna().sum() > 0.1 * len(df):
        issues.append("WARNING: >10% NaN in hybrid sentiment")
    
    # Check date range makes sense
    date_span = (df["ts"].max() - df["ts"].min()).days
    if date_span < 30:
        issues.append("WARNING: Data spans less than 30 days")
    
    # Check drift metrics in reasonable range
    if traj["drift_slope"].abs().max() > 1.0:
        issues.append("WARNING: Extreme drift_slope detected (>1.0 per month)")
    
    # Check for duplicate user-month combinations
    dupes = df.groupby(["user_id", "month"]).size().max()
    if dupes > df.groupby(["user_id", "month"]).size().median() * 10:
        issues.append(f"WARNING: Some user-months have {dupes} reviews (outlier)")
    
    return issues

In [ ]:
if __name__ == "__main__":
    
    logger = setup_logging(RUN_DIR)
    logger.info(f"[run] Starting baseline pipeline: {run_id}")
    
    issues = run_sanity_checks(df_clean, traj)
    for issue in issues:
        logger.warning(issue)

    splits = create_data_splits(traj, monthly, strategy="temporal")
    logger.info(f"[splits] Train: {len(splits['train'])}, Val: {len(splits['val'])}, Test: {len(splits['test'])}")
    
    with open(os.path.join(DATA_DIR, "splits.json"), "w") as f:
        json.dump({k: list(v) for k, v in splits.items()}, f, indent=2)
    
    baselines = compute_baselines(traj, splits)
    logger.info(f"[baseline] Mean drift slope: {baselines['mean_drift_slope']:.4f}")
    
    eval_metrics = evaluate_drift_metrics(traj, splits, baselines)
    metrics.update(eval_metrics)
    
    stat_results = statistical_tests(traj)
    with open(os.path.join(RUN_DIR, "statistical_tests.json"), "w") as f:
        json.dump(stat_results, f, indent=2)
    
    create_diagnostic_plots(traj, monthly, FIG_DIR)
    logger.info(f"[plots] Saved diagnostic plots to {FIG_DIR}")
    
    create_reproducibility_manifest(RUN_DIR, CONFIG, df_clean)
    
    logger.info("[done] Baseline run complete. Check {RUN_DIR} for all outputs.")

2025-11-10 19:47:28,288 - INFO - [run] Starting baseline pipeline: baseline_v1_20251110_183220
2025-11-10 19:47:28,358 - WARNING - WARNING: Extreme drift_slope detected (>1.0 per month)
2025-11-10 19:47:28,360 - WARNING - WARNING: Some user-months have 20 reviews (outlier)
2025-11-10 19:47:28,373 - INFO - [splits] Train: 1859, Val: 398, Test: 400
2025-11-10 19:47:28,386 - INFO - [baseline] Mean drift slope: 0.0005
2025-11-10 19:47:31,873 - INFO - [plots] Saved diagnostic plots to runs\baseline_v1_20251110_183220\figs


NameError: name 'sys' is not defined